In [2]:
import os
import json
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import layers, models
import nltk

# Prepare Corpus

In [4]:
text = open("large_text.txt").read()

nltk.download('punkt_tab')
CORPUS = nltk.sent_tokenize(text)

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\lilyc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [5]:
def build_tokenizer(texts, oov_token="<OOV>", lower=True, num_words=None):
    tok = Tokenizer(oov_token=oov_token, lower=lower, num_words=num_words)
    tok.fit_on_texts(texts)
    return tok

def make_ngram_dataset(tokenizer, texts, min_len=2):
    """
    Builds n-gram sequences for next-word prediction.
    Each sequence[i] is tokens[0:i] -> predict tokens[i]
    """
    sequences = []
    for line in texts:
        tokens = tokenizer.texts_to_sequences([line])[0]
        # create incremental n-grams: [w1,w2]->w3, [w1,w2,w3]->w4, ...
        for i in range(min_len, len(tokens) + 1):
            sequences.append(tokens[:i])
    if not sequences:
        raise ValueError("No sequences constructed; check your corpus/tokenizer.")

    max_len = max(len(s) for s in sequences)
    sequences = pad_sequences(sequences, maxlen=max_len, padding="pre")
    X, y = sequences[:, :-1], sequences[:, -1]
    vocab_size = min(tokenizer.num_words or len(tokenizer.word_index) + 1,
                     len(tokenizer.word_index) + 1)
    return X, y, max_len, vocab_size

# Build Model

In [ ]:
def build_model(vocab_size, seq_len_minus1, embed_dim=128, rnn_units=256, dropout=0.2):
    model = models.Sequential([
        layers.Embedding(input_dim=vocab_size, output_dim=embed_dim, input_length=seq_len_minus1),
        layers.Bidirectional(layers.LSTM(rnn_units, return_sequences=False)),
        layers.Dropout(dropout),
        layers.Dense(rnn_units, activation="relu"),
        layers.Dense(vocab_size, activation="softmax")
    ])
    model.compile(
        loss="sparse_categorical_crossentropy",
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
        metrics=["accuracy"]
    )
    return model

# Inference Helpers

In [7]:
def _apply_temperature(logits, temperature=1.0):
    logits = np.asarray(logits, dtype=np.float64)
    if temperature <= 0:
        # zero (greedy) or negative => fallback to greedy
        one_hot = np.zeros_like(logits)
        one_hot[np.argmax(logits)] = 1.0
        return one_hot
    logits = np.log(np.maximum(logits, 1e-9)) / float(temperature)
    exp = np.exp(logits - np.max(logits))
    return exp / np.sum(exp)

def suggest_next_words(model, tokenizer, seed_text, seq_len_minus1, top_k=5, temperature=1.0):
    """
    Returns top_k candidate next words (no sampling), optionally temperature-adjusted.
    """
    seq = tokenizer.texts_to_sequences([seed_text])[0]
    seq = pad_sequences([seq], maxlen=seq_len_minus1, padding="pre")
    preds = model.predict(seq, verbose=0)[0]
    probs = _apply_temperature(preds, temperature=temperature)

    top_idx = probs.argsort()[-top_k:][::-1]
    idx2word = {idx: w for w, idx in tokenizer.word_index.items() if idx < (tokenizer.num_words or 10**9)}
    return [(idx2word.get(i, "<UNK>"), float(probs[i])) for i in top_idx]

def generate_text(model, tokenizer, seed_text, seq_len_minus1, num_words=10, temperature=1.0, greedy=False):
    """
    Autoregressively generates words. If greedy=True, always pick argmax.
    Otherwise sample by temperature.
    """
    out = seed_text.strip()
    vocab_limit = tokenizer.num_words or (len(tokenizer.word_index) + 1)
    idx2word = {idx: w for w, idx in tokenizer.word_index.items() if idx < vocab_limit}

    for _ in range(num_words):
        seq = tokenizer.texts_to_sequences([out])[0]
        seq = pad_sequences([seq], maxlen=seq_len_minus1, padding="pre")
        preds = model.predict(seq, verbose=0)[0]
        if greedy or temperature <= 0:
            next_id = int(np.argmax(preds))
        else:
            probs = _apply_temperature(preds, temperature=temperature)
            next_id = int(np.random.choice(len(probs), p=probs))
        next_word = idx2word.get(next_id, None)
        if not next_word:
            break
        out += " " + next_word
    return out

# Save/Load

In [8]:
def save_artifacts(model, tokenizer, save_dir="predictive_text_artifacts"):
    os.makedirs(save_dir, exist_ok=True)
    model.save(os.path.join(save_dir, "model.keras"))  # Keras v3 format
    tok_path = os.path.join(save_dir, "tokenizer.pkl")
    with open(tok_path, "wb") as f:
        pickle.dump({
            "config": tokenizer.to_json(),
            "num_words": tokenizer.num_words
        }, f)
    meta = {
        "vocab_size": len(tokenizer.word_index) + 1,
        "num_words_limit": tokenizer.num_words,
    }
    with open(os.path.join(save_dir, "meta.json"), "w") as f:
        json.dump(meta, f, indent=2)
    print(f"Saved model + tokenizer to: {save_dir}")

def load_artifacts(save_dir="predictive_text_artifacts"):
    model = tf.keras.models.load_model(os.path.join(save_dir, "model.keras"))
    with open(os.path.join(save_dir, "tokenizer.pkl"), "rb") as f:
        payload = pickle.load(f)
    tok = Tokenizer.from_json(payload["config"])
    tok.num_words = payload["num_words"]
    with open(os.path.join(save_dir, "meta.json"), "r") as f:
        meta = json.load(f)
    return model, tok, meta

# Train

In [9]:
# tokenize & dataset
tokenizer = build_tokenizer(CORPUS, oov_token="<OOV>", lower=True, num_words=None)
X, y, max_len, vocab_size = make_ngram_dataset(tokenizer, CORPUS, min_len=2)

print(f"Vocab size: {vocab_size}, Max sequence length: {max_len}")

model = build_model(vocab_size=vocab_size, seq_len_minus1=max_len-1,
                    embed_dim=128, rnn_units=256, dropout=0.3)

Vocab size: 4995, Max sequence length: 76


C:\Users\lilyc\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [10]:
history = model.fit(
    X, y,
    epochs=30,
    batch_size=32,
    validation_split=0.2,
    verbose=1,
    # callbacks=[
    #     tf.keras.callbacks.ReduceLROnPlateau(patience=3, factor=0.5, verbose=1),
    #     tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True, monitor="val_loss")
    # ]
)

Epoch 1/30
635/635 ━━━━━━━━━━━━━━━━━━━━ 212s 327ms/step - accuracy: 0.0418 - loss: 7.1213 - val_accuracy: 0.0692 - val_loss: 7.1577
Epoch 2/30
635/635 ━━━━━━━━━━━━━━━━━━━━ 158s 248ms/step - accuracy: 0.0460 - loss: 6.6985 - val_accuracy: 0.0706 - val_loss: 7.2554
Epoch 3/30
635/635 ━━━━━━━━━━━━━━━━━━━━ 168s 264ms/step - accuracy: 0.0592 - loss: 6.4640 - val_accuracy: 0.0863 - val_loss: 7.3955
Epoch 4/30
635/635 ━━━━━━━━━━━━━━━━━━━━ 154s 243ms/step - accuracy: 0.0692 - loss: 6.2286 - val_accuracy: 0.0980 - val_loss: 7.4613
Epoch 5/30
635/635 ━━━━━━━━━━━━━━━━━━━━ 157s 247ms/step - accuracy: 0.0777 - loss: 6.0381 - val_accuracy: 0.0999 - val_loss: 7.6631
Epoch 6/30
635/635 ━━━━━━━━━━━━━━━━━━━━ 182s 287ms/step - accuracy: 0.0827 - loss: 5.8817 - val_accuracy: 0.1060 - val_loss: 7.7715
Epoch 7/30
635/635 ━━━━━━━━━━━━━━━━━━━━ 164s 258ms/step - accuracy: 0.0902 - loss: 5.7248 - val_accuracy: 0.1054 - val_loss: 7.9025
Epoch 8/30
635/635 ━━━━━━━━━━━━━━━━━━━━ 154s 242ms/step - accuracy: 0.0960 -

In [32]:
# quick suggestions
seed = "I am a"
print("\nTop suggestions for next word:")
for word, p in suggest_next_words(model, tokenizer, seed, seq_len_minus1=max_len-1, top_k=5, temperature=0.8):
    print(f"{word:>12s}  {p:.4f}")

# autocomplete demo
generated = generate_text(model, tokenizer, seed, seq_len_minus1=max_len-1, num_words=6, temperature=0.9)
print("\nGenerated:", generated)

# save
save_artifacts(model, tokenizer)


Top suggestions for next word:
      father  0.1171
        real  0.1089
  restaurant  0.0945
         job  0.0824
      little  0.0697

Generated: I am a thought i'd had him some coffee
Saved model + tokenizer to: predictive_text_artifacts
